### 0 Summary

1、使用数据生成方程来拟合回归，要求拟合数据特征之间无多重共线性，否则存在系数偏差

2、使用后门准则来拟合回归，要求拟合数据特征之间无多重共线性，否则存在系数偏差

3、使用后门准则来拟合回归，如果拟合数据特征之间无多重共线性，仍然存在系数偏差，但系数会尽量逼近真实系数

4、如果后门变量只有混淆因子，那么使用后门准则拟合回归明显是不可靠的，因为混淆因子显然与处理变量(treatment)强相关，此时可以考虑PSM等

5、使用后门准则来拟合回归，要求选择的后门变量尽可能远离处理变量（treatment），从而使特征相关性尽可能低

6、使用后门准则来拟合回归，除了使用上述方法，或许还可以寻找一个参数估计算法，这个方法不受特征相关性的影响

7、使用后门准则来拟合回归，如果存在数据特征之间的多重共线性，那么一个变量的系数估计将是整合系数效应

### 1 causal-learn

In [ ]:
import numpy as np

def gen_coef():
    return np.random.uniform(1, 3)

np.random.seed(0)
sample_size, loc, scale = 1000, 0.0, 1.0
L1 = np.random.normal(loc=loc, scale=scale, size=sample_size)
L2 = np.random.normal(loc=loc, scale=scale, size=sample_size)
L3 = np.random.normal(loc=loc, scale=scale, size=sample_size)
X1 = gen_coef() * L1 + gen_coef() * L2 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X2 = gen_coef() * X1 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X3 = gen_coef() * X2 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X4 = gen_coef() * X1 + gen_coef() * L3 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X5 = gen_coef() * X3 + gen_coef() * L3 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X6 = gen_coef() * L1 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X7 = gen_coef() * L2 + gen_coef() * L3 + gen_coef() * X6 + np.random.normal(loc=loc, scale=scale,
                                                                            size=sample_size)
data = np.array([L1,L2,L3,X1, X2, X3, X4, X5, X6, X7]).T
data

### 2 chaoxi

In [33]:
import numpy as np
import pandas as pd

N = 100000
loc, scale = 0, 1

IQ = np.random.normal(loc=loc, scale=scale, size=sample_size)
edu = 0.5 * IQ + np.random.normal(loc=loc, scale=scale+1, size=sample_size)
earnings = 0.3 * IQ + 0.4 * edu + np.random.normal(loc=loc, scale=scale+2, size=sample_size)
art = 1.2 * edu + 0.6 * earnings + np.random.normal(loc=loc, scale=scale+3, size=sample_size)

data = pd.DataFrame({'IQ':IQ, 'edu':edu, 'earnings':earnings, 'art':art})

In [34]:
import statsmodels.formula.api as smf

model = smf.ols(formula='earnings ~ IQ + edu', data=data)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     65.27
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           2.30e-27
Time:                        13:55:56   Log-Likelihood:                -2524.9
No. Observations:                1000   AIC:                             5056.
Df Residuals:                     997   BIC:                             5071.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0507      0.096     -0.529      0.5

In [35]:
import statsmodels.formula.api as smf

model = smf.gls(formula='earnings ~ IQ + edu', data=data)
result = model.fit()
print(result.summary())

                            GLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.116
Model:                            GLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     65.27
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           2.30e-27
Time:                        13:55:56   Log-Likelihood:                -2524.9
No. Observations:                1000   AIC:                             5056.
Df Residuals:                     997   BIC:                             5071.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0507      0.096     -0.529      0.5

In [36]:
import statsmodels.formula.api as smf

model = smf.ols(formula='earnings ~ IQ + edu + art', data=data)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     137.7
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           1.30e-74
Time:                        13:55:56   Log-Likelihood:                -2413.0
No. Observations:                1000   AIC:                             4834.
Df Residuals:                     996   BIC:                             4854.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0028      0.086      0.033      0.9

In [37]:
import statsmodels.formula.api as smf

model = smf.gls(formula='earnings ~ IQ + edu + art', data=data)
result = model.fit()
print(result.summary())

                            GLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.293
Model:                            GLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     137.7
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           1.30e-74
Time:                        13:55:56   Log-Likelihood:                -2413.0
No. Observations:                1000   AIC:                             4834.
Df Residuals:                     996   BIC:                             4854.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0028      0.086      0.033      0.9

gls 和 ols 的系数拟合是一样的，这可能是因为没有异方差的问题？但是现在每个特征的方差都不一样，结果还是之前的结论

当R和python的结果不一样时，应该选择哪个更合适？

### 3 xingming

In [75]:
import numpy as np
import pandas as pd

N = 100000
loc, scale = 0, 1

C1 = np.random.normal(loc=loc, scale=scale, size=sample_size)
C2 = C1 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C3 = C2 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C4 = C3 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C5 = C4 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C6 = C5 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X1 = np.random.normal(loc=loc, scale=scale, size=sample_size)
X2 = np.random.normal(loc=loc, scale=scale, size=sample_size)
B = C1 + C2 + C3 + C4 + C5 + C6
A = B + C1 + C2 + C3 + C4 + C5 + C6 + X1 + X2

X3 = np.random.normal(loc=loc, scale=scale, size=sample_size)
X4 = np.random.normal(loc=loc, scale=scale, size=sample_size) + B 
E = B + X3 + X4

X5 = C6 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X6 = np.random.normal(loc=loc, scale=scale, size=sample_size)
F = B + X5 + X6

data = pd.DataFrame({'C1':C1, 'C2':C2, 'C3':C3, 'C4':C4, 'C5':C5, 'C6':C6,
                     'X1':X1, 'X2':X2, 'X3':X3, 'X4':X4, 'X5':X5, 'X6':X6,
                     'A':A, 'B':B, 'E':E, 'F':F})

#### 3.1 对B进行回归

In [76]:
import statsmodels.formula.api as smf

model = smf.ols(formula='B ~ C1 + C2 + C3 + C4 + C5 + C6', data=data) ### 直接拟合数据生成方程 （准！) 无多重共线性
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      B   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.360e+33
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                 31916.
No. Observations:                1000   AIC:                        -6.382e+04
Df Residuals:                     993   BIC:                        -6.378e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.124e-15   1.06e-16    -10.586      0.0

#### 3.2 对A进行回归

In [77]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B + C1 + C2 + C3 + C4 + C5 + C6', data=data) ### 仅考虑后门变量 但存在多重共线性
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 3.041e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                -1762.9
No. Observations:                1000   AIC:                             3540.
Df Residuals:                     993   BIC:                             3574.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0152      0.045      0.339      0.7

In [78]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B', data=data) ### 解决多重共线性  （准！）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 1.821e+05
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                -1766.3
No. Observations:                1000   AIC:                             3537.
Df Residuals:                     998   BIC:                             3547.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0175      0.045      0.390      0.6

In [79]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B + C1 + C2 + C3 + C4 + C5 + C6 + X1 + X2', data=data) ### 直接拟合数据生成方程 （不准）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.722e+33
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                 31482.
No. Observations:                1000   AIC:                        -6.295e+04
Df Residuals:                     991   BIC:                        -6.290e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.608e-16   1.64e-16     -2.201      0.0

In [80]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B + C1 + C2 + C3 + C4 + C5', data=data)  ### 当后门变量未充分考虑时
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 3.041e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                -1762.9
No. Observations:                1000   AIC:                             3540.
Df Residuals:                     993   BIC:                             3574.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0152      0.045      0.339      0.7

#### 3.3 对E进行回归

In [81]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='E ~ B', data=data)  ### 仅考虑后门变量 但后门变量为空时 （准！）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      E   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 1.870e+05
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                -1752.5
No. Observations:                1000   AIC:                             3509.
Df Residuals:                     998   BIC:                             3519.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0489      0.044      1.104      0.2

In [82]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='E ~ B + X3 + X4', data=data)  ### 直接拟合数据生成方程 （准！) 无多重共线性
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      E   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.382e+33
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                 31560.
No. Observations:                1000   AIC:                        -6.311e+04
Df Residuals:                     996   BIC:                        -6.309e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.274e-16   1.51e-16      3.487      0.0

#### 3.4 对F进行回归

In [83]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='F ~ B + X6', data=data)  ### 仅考虑后门变量 且后门变量为1个时 （准？）照理说应该不准，B和X6存在共线性
result = model.fit()  ### 可能是因为共线性较弱？
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      F   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                 3.122e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                -1818.4
No. Observations:                1000   AIC:                             3643.
Df Residuals:                     997   BIC:                             3657.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0289      0.047      0.610      0.5

In [84]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='F ~ B + X6 + X7', data=data)  ### 直接拟合数据生成方程 （不准）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      F   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                 2.082e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:04:02   Log-Likelihood:                -1817.7
No. Observations:                1000   AIC:                             3643.
Df Residuals:                     996   BIC:                             3663.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0325      0.047      0.684      0.4